# Libraries

In [1]:
import os
from crewai import Agent, Task, Crew, Process
import crewai
import pandas as pd
from crewai_tools import tool
from dotenv import load_dotenv
from crewai import LLM

In [3]:
pip show crewai

Name: crewaiNote: you may need to restart the kernel to use updated packages.

Version: 0.63.2
Summary: Cutting-edge framework for orchestrating role-playing, autonomous AI agents. By fostering collaborative intelligence, CrewAI empowers agents to work together seamlessly, tackling complex tasks.
Home-page: https://crewai.com
Author: Joao Moura
Author-email: joao@crewai.com
License: 
Location: c:\Users\VICTUS\AppData\Local\Programs\Python\Python311\Lib\site-packages
Requires: appdirs, auth0-python, click, embedchain, instructor, json-repair, jsonref, langchain, litellm, openai, opentelemetry-api, opentelemetry-exporter-otlp-proto-http, opentelemetry-sdk, poetry, pydantic, python-dotenv, regex
Required-by: 


# LLM setting

In [4]:
load_dotenv()
gpt = LLM(
    model="gpt-4o-mini-2024-07-18",
    temperature=0.0,
    top_p=0.1,
    api_key=os.getenv("OPENAI_API_KEY")
)

# Tools

## Tool for Financial Analyst

In [6]:
class CSVFilterTool:
    def __init__(self, csv_file_path="news_headline_final_2.csv"):
        self.csv_file_path = csv_file_path

    def filter_by_month_and_year(self, month: int, year: int):
        df = pd.read_csv(self.csv_file_path, parse_dates=['date'])
        filtered_df = df[(df['date'].dt.year == year) &
                         (df['date'].dt.month == month)]
        filtered_data = filtered_df[['news_headline', 'stock_symbol']]
        return filtered_data


def filter_2(filtered_data):
    """
    After filtering the news headlines for a specific month and year, organize the data to be stored in a structured format.

    Args:
        filtered_data (pd.DataFrame): A Data containing the filtered news headlines and stock symbols.

    Returns:
        str: A Data containing the filtered news headlines and stock symbols for a specific month and year in a structured format. 
    """
    results = {}
    for stock_symbol, group in filtered_data.groupby('stock_symbol'):
        num_headlines = len(group)
        titles_combined = ",".join(group['news_headline'].tolist())
        results[stock_symbol] = {
            'count': num_headlines,
            'titles': titles_combined
        }
    result2 = ""
    for stock_symbol, result in results.items():
        result2 += f"Stock: {stock_symbol}\nNumber of News Headlines: {result['count']}\nNews Headlines: {result['titles']}\n\n"
    return result2


@tool
def load_data_for_month(month: int, year: int) -> str:
    """
    Load a filtered news headlines in a structured format that associated stock symbols from a CSV file for a specific month and year.

    Args:
        month (int): The month to filter by (1-12).
        year (int): The year to filter by.

    Returns:
        str: A string representation of the filtered news headlines in a structured format for a specific month and year.
    """

    csv_tool = CSVFilterTool("news_headline_final_2.csv")

    filtered_data = csv_tool.filter_by_month_and_year(month, year)

    filter_result = filter_2(filtered_data)

    return filter_result

## Tool for Risk Advisor

In [7]:
@tool
def gather_stock_details(stocks: str) -> str:
    """
    Gather stock details (Stocks Names, Annual Return and Annual Standard Deviation) for the given stock names.

    Args:
    stocks (str): Comma-separated stock names as input, e.g. 'stock a', 'stock b', 'stock c'

    Returns:
    str: The details (Stocks Names, Annual Return and Annual Standard Deviation) of the stocks.
    """
    data = pd.read_csv(
        "risk_historical_data_annual.csv")

    stock_list = [stock.strip() for stock in stocks.split(',')]
    print(stock_list)
    # filter selected stocks
    filtered_data = data[data['Stocks Names'].isin(stock_list)]

    if filtered_data.empty:
        return "No data found for the provided stock names."

    result = ""
    for index, row in filtered_data.iterrows():
        result += (
            f"{row['Stocks Names']}, "
            f"Annual Return: {row['Annual Return']}, "
            f"Annual Standard Deviation: {row['Annual Standard Deviation']}\n"
        )

    return result

# Agent, Task, Crew

## Financial Analyst

In [8]:
financial_analyst_agent = Agent(
    role='Financial Analyst',
    goal="""
    - Using extensive experience in the Thai financial market, analyze news headlines about Thai stocks in the Thai financial market from the data gathered using the tool: [load_data_for_month]. The goal is to identify investment opportunities based on the most positive sentiment expressed in these headlines. Identify the top 10 stocks to consider for investment next month, as this month is {month}.
    """,
    verbose=True,
    backstory="""
    - Specialist with extensive experience in the Thai financial market.
    - Expert with a deep understanding of financial knowledge and analytical skills in the Thai financial market.
    - Experienced in analyzing stock news headlines to assess sentiment and recommend top-performing stocks.
    - Expert in the Thai language, enabling accurate interpretation and analysis of news headlines.
""",
    tools=[load_data_for_month],
    llm=gpt
    ,memory = False
)

financial_analyst_task = Task(
    description="""
    - Analyze news headlines to identify investment opportunities by selecting the top 10 stocks with the most positive sentiment in their headlines from the data gathered using the tool: [load_data_for_month]. This data will provide stock news headlines for the month of {month} and the year {year}. Utilize your financial knowledge and analytical skills in the Thai financial market to gain insights into these investment opportunities.
    - Example of data details:
        Stock: Stock A → Name of the stock.
        Number of News Headlines: n → The number of news headlines related to the stock mentioned above.
        News Headlines: → A list of news headlines related to the stock mentioned above (from latest to oldest within the foling month and year).
    """,

    expected_output="""
    - A report on the top 10 stocks to consider for investment next month, based on an analysis of stock news headlines from the data gathered using the tool: [load_data_for_month]. The selection is focused on identifying the top stocks with the most positive sentiment in their headlines. 
    - The report must include 3 things:
        1. Reason for Selection: An explanation of why each of the top 10 stocks was chosen and provide the insights derived from the news headlines for each selected stock, emphasizing the positive sentiment.
        2. Conclusion: A list of the selected top 10 stocks, formatted exactly as 'Stock 1', 'Stock 2'.
        - Report format:
            1. Stock 1
                Reason for Selection: [Explanation]
            2. Stock 2
                Reason for Selection: [Explanation]
            Conclusion: 'Stock 1', 'Stock 2'

""",
    agent=financial_analyst_agent
)

## Risk Advisor

In [9]:
from crewai import Crew, Process

risk_advisor_agent = Agent(
    role='Risk Advisor',
    goal="""
    - Utilizing extensive financial market experience and statistical skills to analyze historical stock data from the stock list provided by the financial_analyst_agent using the tool: [gather_stock_details]. Leverage the Average Annual Return and Annual Standard Deviation of stocks, as gathered by the tool, to identify the top 5 stocks with the {level} risk level.
    """,
    verbose=True,
    backstory="""
    - Specialist with extensive experience in the Thai financial market.
    - Expert with a deep understanding of financial knowledge and analytical skills in the Thai financial market.
    - Specialist in statistics, especially in finance.
""",
    llm=gpt 
    , memory=False,
    tools=[gather_stock_details]


)

risk_advisor_task = Task(
    description="""
    - Analyze the historical stock data of the stock list provided by the financial_analyst_agent. Using the provided Average Annual Return and Annual Standard Deviation and your experience in the financial market and statistical skills, assess the risk of the stock list provided by the financial_analyst_agent, and identify the top 5 stocks with the most {level} of risk for investment. 
    - Example of data details:
        1. The "Stock Names" contains the names of each stock.
        2. The "Average Annual Return" contains the average annual return for each stock based on its historical stock prices from 3/3/2023 to 9/29/2023.
        3. The "Annual Standard Deviation" contains the annual standard deviation for each stock based on its historical stock prices from 3/3/2023 to 9/29/2023.
        - Some stock names might be similar to each other, so it is important to be cautious and ensure that you query or use the correct information for each stock.
    - Utilize your financial knowledge, analytical skills, and statistical expertise in the financial market to analyze the historical stock data of the stock list provided by the financial_analyst_agent. Identify the top 5 stocks with the most {level} risk.
    """,

    expected_output="""
    - A report on the top 5 stocks selecting from the stock list provided by the financial_analyst_agent to consider as having the most {level} risk, based on an analysis of historical stock data from the data gathered using the tool.
    - The report must include 2 things:
        1. Reason for Selection: An in-depth explanation detailing why each of the top 5 stocks was chosen as having the most {level} risk, using data from the data gathered using the tool to like provided Average Annual Return and Annual Standard Deviation to support your explanation. 
        2. Conclusion: A list of the selected top 5 stocks with the most {level} risk, formatted exactly as 'Stock 1', 'Stock 2'.
        - Report format:
            1. Stock 1
                Reason for Selection: [An in-depth explanation]
            2. Stock 2
                Reason for Selection: [An in-depth explanation]
            Conclusion: 'Stock 1', 'Stock 2'
""",
    agent=risk_advisor_agent
)

## Crew

In [10]:
crew = Crew(
    agents=[financial_analyst_agent, risk_advisor_agent],
    tasks=[financial_analyst_task, risk_advisor_task],
    process=Process.sequential
)

## Usage (Example)

- using news 10/2023 - 10/2024
- to predict 11/2023 - 11/2024

In [ ]:
from datetime import datetime
now = datetime.now()
current_time = now.strftime("%Y-%m-%d %H:%M:%S")

print("Run at", current_time)
result = crew.kickoff(inputs={'year': 2023,
                              'month': 10,
                              'level': 'high'
                              })

print(result)

In [ ]:
from datetime import datetime
now = datetime.now()
current_time = now.strftime("%Y-%m-%d %H:%M:%S")

print("Run at", current_time)
result = crew.kickoff(inputs={'year': 2023,
                              'month': 10,
                              'level': 'low'
                              })

print(result)